In [1]:
import json
import os
import openai
from dotenv import load_dotenv
from minsearch import Index

load_dotenv()

documents = []

with open('documents.json', 'r') as file:

    docs = json.load(file)
    
    

In [2]:
for doc in docs:

    for document in doc['documents']:

        document['course'] = doc['course'] 
        documents.append(document)

In [3]:
textfields = ["text", "section", "question"]


indobject = Index(text_fields = textfields, keyword_fields = ['course'])
indobject.fit(documents)

In [4]:


def search(query, boost_dict = {"question": 3}, filter_dict =  {"course":"mlops-zoomcamp"} ):

    
    
    
    context = indobject.search(query = query, boost_dict = boost_dict, filter_dict = filter_dict
                    )
    
    return context

In [6]:
def build_prompt(query, related_docs):
    
    
    
    
    prompt = """
    You are an assistant for teaching online courses answer the question based on the context provided.
    Only the information available in the provided context should be used strictly adhere to this
    
    question:{query}
    
    context:{context}
    
    
    """.strip()
    
    
    context = ""
    for doc in related_docs:
        
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']} \n\n"
        
    
    prompt = prompt.format(query = query, context = context).strip()
    
    return prompt

In [16]:
def chat(prompt, model = "phi3"):
   
    client = openai.OpenAI(
        base_url = "http://localhost:11434/v1/",
        
        api_key = "ollama"
    )
    
    response = client.chat.completions.create(
        model = model,
        messages = [{"role":"system", "content":"you are a faq assistant"},
                    {"role": "user", "content": prompt}]
    )
    
    print(response)
    
    return response.choices[0].message.content
    
    

In [17]:
def rag(query, boost_dict = {"question": 3}, course_filter =  {"course": "data-engineering-zoomcamp"}, model = "phi3"):
    
    context = search(query = query, boost_dict = boost_dict, filter_dict = course_filter)
    
    prompt = build_prompt(query, context)
    
    answer = chat(prompt, model = model)
    
    return answer

In [18]:
rag(query = "How to run spark engine ?")

ChatCompletion(id='chatcmpl-196', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To set Python as a startup on Windows, you can add it in `startup` folder within the users\' path directories. Here is how to properly configure this setting for multiple common cases when starting up both local applications and Spark:\n\nFor **Local Applications** using Anaconda (Python-based):\n```batchfile\nC:\\Users\\[username]\\anaconda3_20*b*64*\\Scripts> echo %PYTHONHOME%\n<empty> # Often you might get \'NULL\', not set.\n\nREM Set PYTHONPATH to work with libraries/packages under your local Anaconda environment directory and add it as a startup command after setting the path for convenience:\nsetx SZ_USERPY THISUSERDIR "%UserProfile%\\AppData\\Local\\<username>\\Anaconda3_20*b*64\\" >NUL /M 1>NUL || set "SZ_USERPY=%AppData%\\Local\\<userprofilename>>";\nsetx SZ_STARTUP %APPDATA%/Startup/%SystemRoot%/system32;C:\\Users\\\\[username]\\.anaconda3_20

'To set Python as a startup on Windows, you can add it in `startup` folder within the users\' path directories. Here is how to properly configure this setting for multiple common cases when starting up both local applications and Spark:\n\nFor **Local Applications** using Anaconda (Python-based):\n```batchfile\nC:\\Users\\[username]\\anaconda3_20*b*64*\\Scripts> echo %PYTHONHOME%\n<empty> # Often you might get \'NULL\', not set.\n\nREM Set PYTHONPATH to work with libraries/packages under your local Anaconda environment directory and add it as a startup command after setting the path for convenience:\nsetx SZ_USERPY THISUSERDIR "%UserProfile%\\AppData\\Local\\<username>\\Anaconda3_20*b*64\\" >NUL /M 1>NUL || set "SZ_USERPY=%AppData%\\Local\\<userprofilename>>";\nsetx SZ_STARTUP %APPDATA%/Startup/%SystemRoot%/system32;C:\\Users\\\\[username]\\.anaconda3_20*b*64*\\;SZ_USERPY >>NUL 1>NUL || set "SZ_STARTUP=%UserProfile%.local\\startup"\ncd /d %SZ_STARTUP%\nif not "%CD%"=="%~dp0" cd /d %~dp

In [19]:
print(_)

To set Python as a startup on Windows, you can add it in `startup` folder within the users' path directories. Here is how to properly configure this setting for multiple common cases when starting up both local applications and Spark:

For **Local Applications** using Anaconda (Python-based):
```batchfile
C:\Users\[username]\anaconda3_20*b*64*\Scripts> echo %PYTHONHOME%
<empty> # Often you might get 'NULL', not set.

REM Set PYTHONPATH to work with libraries/packages under your local Anaconda environment directory and add it as a startup command after setting the path for convenience:
setx SZ_USERPY THISUSERDIR "%UserProfile%\AppData\Local\<username>\Anaconda3_20*b*64\" >NUL /M 1>NUL || set "SZ_USERPY=%AppData%\Local\<userprofilename>>";
setx SZ_STARTUP %APPDATA%/Startup/%SystemRoot%/system32;C:\Users\\[username]\.anaconda3_20*b*64*\;SZ_USERPY >>NUL 1>NUL || set "SZ_STARTUP=%UserProfile%.local\startup"
cd /d %SZ_STARTUP%
if not "%CD%"=="%~dp0" cd /d %~dp0 && title StartUp & your python

In [ ]:
query = "How to run spark engine ?"

In [ ]:
context = search(query = query, filter_dict = {"course": "data-engineering-zoomcamp"})

In [ ]:
context

[{'text': 'Change the working directory to the spark directory:\nif you have setup up your SPARK_HOME variable, use the following;\ncd %SPARK_HOME%\nif not, use the following;\ncd <path to spark installation>\nCreating a Local Spark Cluster\nTo start Spark Master:\nbin\\spark-class org.apache.spark.deploy.master.Master --host localhost\nStarting up a cluster:\nbin\\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost',
  'section': 'Module 5: pyspark',
  'question': 'How to spark standalone cluster is run on windows OS',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'The docker will keep on crashing continuously\nNot working after restart\ndocker engine stopped\nAnd failed to fetch extensions pop ups will on screen non-stop\nSolution :\nTry checking if latest version of docker is installed / Try updating the docker\nIf Problem still persist then final solution is to reinstall docker\n(Just have to fetch images again else no issues)',
  'sectio

In [ ]:
prompt = build_prompt(query = query, related_docs = context )

In [ ]:
answer = chat(prompt = prompt)

In [ ]:
answer

"To run a Spark standalone cluster on Windows OS, follow these steps: \n\n1. Change the working directory to the Spark directory:\n```\ncd %SPARK_HOME%\n```\nIf you haven't set up the `SPARK_HOME` variable, use the path to your Spark installation instead. \n\n2. Start the Spark Master by navigating to the bin folder:\n```\nbin\\spark-class org.apache.spark.deploy.master.Master --host localhost\n```\n\n3. Start the worker nodes by providing the Spark Master URL: \n```\nbin\\spark-class org.apache.spark.deploy.worker.Worker spark://localhost:7077 --host localhost\n```\n\nMake sure to replace `localhost:7077` with the appropriate IP address and port if your setup requires a different configuration. \n\nThese steps will help you run the Spark engine in standalone mode on a Windows operating system."